# Model KNN 

In [2]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB 

In [3]:
# load basic libraries
import pandas as pd
import numpy as np
import mne
from pathlib import Path
import pickle
import time

from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

# set directories
# %cd D:/Programy/Anaconda3/Projects/EEG ML project # working directory
%cd D:
pkls = './Pickles/' # objects & variables

D:\Programy\Anaconda3\Projects\EEG ML project


In [4]:
# to see all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# load split sets, eeg only
with open(pkls +'xy_train.pkl', 'rb') as handle:
    x_train = pickle.load(handle)
    y_train = pickle.load(handle)
y_train.shape
x_train.shape

(6040, 1)

(6040, 94)

In [12]:
# fit
cv = 10
param_grid = {'n_neighbors': [3, 5, 8, 11, 15, 19], 
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

scorers = {'accuracy': make_scorer(accuracy_score),
           'f1'      : make_scorer(f1_score, average = 'weighted')}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit='accuracy', verbose=1, 
                        scoring = scorers, return_train_score=True, cv = cv, n_jobs = -1)

grid.fit(x_train, y_train.values.ravel())
best_knn = grid.best_estimator_

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  3.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 8, 11, 15, 19],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=weighted)},
             verbose=1)

In [53]:
# evaluate and save
best_knn_res = round(pd.DataFrame(index = ['knn'], data  = 
          {'acc mean':[grid.cv_results_['mean_test_accuracy'][grid.best_index_]],
           'acc 2sd': [2*grid.cv_results_['std_test_accuracy'][grid.best_index_]],
           'f1 mean': [grid.cv_results_['mean_test_f1'][grid.best_index_]],
           'f1 2sd' : [2*grid.cv_results_['std_test_f1'][grid.best_index_]]}),4)
best_knn_res

,acc mean,acc 2sd,f1 mean,f1 2sd
knn,0.5194,0.1202,0.5028,0.1075


In [51]:
# train set score for comparison
best_knn.fit(x_train, y_train.values.ravel())
print("training acc mean: {:.4f}".format(
    best_knn.score(x_train, y_train)))
print("training f1  mean: {:.4f}".format(
    f1_score(y_train, best_knn.predict(x_train), average = 'weighted')))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=19, p=2,
                     weights='uniform')

training acc mean: 0.6765
training f1  mean: 0.6730


In [54]:
# save 
with open('./Pickles/best_knn.pkl', 'wb') as handle:
    pickle.dump(best_knn, handle)
    pickle.dump(best_knn_res, handle)

In [11]:
with open('./Pickles/best_knn.pkl', 'rb') as handle:
    best_knn = pickle.load(handle)
#     best_knn_res = pickle.load(handle)

EOFError: Ran out of input

# Model Naive Bayes 

In [56]:
# fit
gnb = GaussianNB().fit(x_train, y_train.values.ravel()) 

# evaluate
cv = 10
scores = cross_val_score(gnb, x_train, y_train.values.ravel(), cv=cv)
scoresf1 = cross_val_score(gnb, x_train, y_train.values.ravel(), cv=cv, scoring='f1_weighted')

gnb_res = pd.DataFrame(data = [round(scores.mean(),3), round(scores.std() * 2, 3), 
                               round(scoresf1.mean(),3), round(scoresf1.std() * 2,3)]).T
gnb_res.columns = ['acc mean', 'acc 2sd', 'f1 mean','f1 2sd']
gnb_res.index = ['gnb']
gnb_res

,acc mean,acc 2sd,f1 mean,f1 2sd
gnb,0.543,0.156,0.524,0.179


In [57]:
# save best model
with open('./Pickles/best_gnb.pkl', 'wb') as handle:
    pickle.dump(gnb, handle)
    pickle.dump(gnb_res, handle)

In [47]:
from sklearn.metrics import roc_auc_score

y_prob = gnb.predict_proba(x_train)
type(y_prob)
type(y_train.values.ravel())
sum(y_prob)

macro_roc_auc_ovo = roc_auc_score(y_true = y_train.values.ravel(), y_score = y_prob, multi_class="ovo", average="macro")
# weighted_roc_auc_ovo = roc_auc_score(y_train.values.ravel(), y_prob, multi_class="ovo", average="weighted")
# macro_roc_auc_ovr = roc_auc_score(y_train.values.ravel(), y_prob, multi_class="ovr", average="macro")
# weighted_roc_auc_ovr = roc_auc_score(y_train.values.ravel(), y_prob, multi_class="ovr", average="weighted")

# print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
#       "(weighted by prevalence)"
#       .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
# print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
#       "(weighted by prevalence)"
#       .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

numpy.ndarray

numpy.ndarray

array([675.1796559, 655.8203441])

ValueError: bad input shape (1331, 2)